In [1]:
'''

This code is part of the SIPN2 project focused on improving sub-seasonal to seasonal predictions of Arctic Sea Ice. 
If you use this code for a publication or presentation, please cite the reference in the README.md on the
main page (https://github.com/NicWayand/ESIO). 

Questions or comments should be addressed to nicway@uw.edu

Copyright (c) 2018 Nic Wayand

GNU General Public License v3.0


'''

import xarray as xr
import os
import numpy as np

In [2]:
grid_dir = '/home/disk/sipn/nicway/data/grids'

In [3]:
ds_region = xr.open_dataset(os.path.join(grid_dir, 'sio_2016_mask.nc'))
ds_region.set_coords(['lat','lon'], inplace=True);
# Note minor -0.000004 degree differences in latitude
# Replace with nsdidc obs file lat/lon
ds_nsidc = xr.open_dataset('/home/disk/sipn/nicway/data/obs/NSIDC_0081/sipn_nc/nt_20170804_f18_nrt_n.nc')
ds_region['lat'] = ds_nsidc.lat
ds_region['lon'] = ds_nsidc.lon
# Clean up string (need to do at import)
ds_region.coords['region_names'] = xr.DataArray([x.item(0).decode("utf-8").strip() for x in ds_region.region_names],
                                                dims='nregions', coords={'nregions':ds_region.nregions})
ds_region.rename({'nx':'x', 'ny':'y'}, inplace=True);
ds_region['nregions'] = np.arange(1,22)
rm_regs = ['Ice-free Oceans', 'null', 'land outline', 'land' ]
# Get regions index for ones we want
ds_region.coords['ocean_regions']  = xr.DataArray(ds_region.where(~ds_region.region_names.isin(rm_regs), drop=True).nregions.values,
                                                dims='ocean_regions')
ds_region.area.attrs = '' # was complaining about coords in attrs so remove

# Fix greenland typo
old_names = ds_region.region_names.values
new_names = [w.replace('East Greeland Sea', 'East Greenland Sea') for w in old_names]
ds_region['region_names'] = xr.DataArray(new_names, coords=ds_region.region_names.coords, 
                                         dims=ds_region.region_names.dims)
# drop time
ds_region = ds_region.drop('time')
ds_region.to_netcdf(os.path.join(grid_dir,'sio_2016_mask_Update.nc'))

/home/disk/sipn/nicway/anaconda3/envs/esio/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: The inplace argument has been deprecated and will be removed in xarray 0.12.0.
  
/home/disk/sipn/nicway/anaconda3/envs/esio/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: The inplace argument has been deprecated and will be removed in xarray 0.12.0.
  # This is added back by InteractiveShellApp.init_path()


In [4]:
ds_region.region_names

<xarray.DataArray 'region_names' (nregions: 21)>
array(['Ice-free Oceans', 'Sea of Okhotsk', 'Bering', 'Hudson Bay', 'St John',
       'Baffin Bay', 'East Greenland Sea', 'Barents Sea', 'Kara Sea',
       'Laptev Sea', 'East Siberian Sea', 'Chukchi Sea', 'Beaufort Sea',
       'Canadian Islands', 'Central Arctic', 'null', 'null', 'null', 'null',
       'land', 'land outline'], dtype='<U18')
Coordinates:
    region_names  (nregions) <U18 'Ice-free Oceans' ... 'land outline'
  * nregions      (nregions) int64 1 2 3 4 5 6 7 8 9 ... 14 15 16 17 18 19 20 21

In [5]:
list(zip(ds_region.region_names.sel(nregions=ds_region.ocean_regions.values).values, 
         ds_region.region_names.sel(nregions=ds_region.ocean_regions.values).nregions.values))

[('Sea of Okhotsk', 2),
 ('Bering', 3),
 ('Hudson Bay', 4),
 ('St John', 5),
 ('Baffin Bay', 6),
 ('East Greenland Sea', 7),
 ('Barents Sea', 8),
 ('Kara Sea', 9),
 ('Laptev Sea', 10),
 ('East Siberian Sea', 11),
 ('Chukchi Sea', 12),
 ('Beaufort Sea', 13),
 ('Canadian Islands', 14),
 ('Central Arctic', 15)]

In [6]:
ds_region.region_names

<xarray.DataArray 'region_names' (nregions: 21)>
array(['Ice-free Oceans', 'Sea of Okhotsk', 'Bering', 'Hudson Bay', 'St John',
       'Baffin Bay', 'East Greenland Sea', 'Barents Sea', 'Kara Sea',
       'Laptev Sea', 'East Siberian Sea', 'Chukchi Sea', 'Beaufort Sea',
       'Canadian Islands', 'Central Arctic', 'null', 'null', 'null', 'null',
       'land', 'land outline'], dtype='<U18')
Coordinates:
    region_names  (nregions) <U18 'Ice-free Oceans' ... 'land outline'
  * nregions      (nregions) int64 1 2 3 4 5 6 7 8 9 ... 14 15 16 17 18 19 20 21